In [ ]:
import struct
import sys
from threading import Thread

import pyaudio
from picovoice import Picovoice

import spotipy
from spotipy.oauth2 import SpotifyOAuth

import speech_recognition as sr
from pydub import AudioSegment
from pydub.playback import play

import os
from google.cloud import language_v1 as language

from apa102 import APA102
from gpiozero import LED

COLORS_RGB = dict(
    off=(0, 0, 0),
    blue=(0, 0, 255),
    green=(0, 255, 0),
    orange=(255, 128, 0),
    pink=(255, 51, 153),
    purple=(128, 0, 128),
    red=(255, 0, 0),
    white=(255, 255, 255),
    yellow=(255, 255, 51),
)

# Inicializacion LEDs
driver = APA102(num_led=12)
power = LED(5)
power.on()

class PicovoiceDemo(Thread):
    def __init__(
            self,
            keyword_path,
            context_path,
            porcupine_sensitivity=0.75,
            rhino_sensitivity=0.25):
        super(PicovoiceDemo, self).__init__()

        def inference_callback(inference):
            return self._inference_callback(inference)

        # Creacion objeto picovoice
        self._picovoice = Picovoice(
            keyword_path=keyword_path,
            wake_word_callback=self._wake_word_callback,
            context_path=context_path,
            inference_callback=inference_callback,
            porcupine_sensitivity=porcupine_sensitivity,
            rhino_sensitivity=rhino_sensitivity)

        self._context = self._picovoice.context_info

        # Saludo
        self._set_color(COLORS_RGB['purple'])
        voz = AudioSegment.from_mp3("saludo.mp3")
        play(voz)
        self._set_color(COLORS_RGB['off'])

        # Se crea el objeto Spotify
        self.sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
                                    client_secret="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
                                    redirect_uri="http://polaris/callback",
                                    scope="user-read-private user-read-playback-state user-modify-playback-state"))
        
        # Se crea el objeto Natural Language
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="google-credentials.json"
        self.nl = language.LanguageServiceClient()

    @staticmethod
    def _set_color(color):
        for i in range(12):
            driver.set_pixel(i, color[0], color[1], color[2])
        driver.show()

    @staticmethod
    def _speech_to_text():
        text = None
        r = sr.Recognizer()  
        with sr.Microphone() as source:  
            print("Dime algo:")  
            audio = r.listen(source)  
            text = r.recognize_google(audio)  
        print("Has dicho: {}".format(text))
        return text

    # Se llama al metodo cuando se entiende la wake word
    @staticmethod
    def _wake_word_callback():
        print('[wake word]\n')
        PicovoiceDemo._set_color(COLORS_RGB['purple'])

    # Operaciones seg\u00fan las orden entendida
    def _inference_callback(self, inference):
        print('{')
        print("  is_understood : '%s'," % 'true' if inference.is_understood else 'false')
        if inference.is_understood:
            print("  intent : '%s'," % inference.intent)
            if len(inference.slots) > 0:
                print('  slots : {')
                for slot, value in inference.slots.items():
                    print("    '%s' : '%s'," % (slot, value))
                print('  }')
        print('}\n')

        if inference.is_understood:
            if inference.intent == 'cancion':
                # Se pausa la reproduccion
                self.sp.pause_playback() 
                
                # Se pregunta al usuario la cancion
                voice = AudioSegment.from_mp3("cancion.mp3")
                play(voice)

                # Se obtiene la cancion
                song = self._speech_to_text()  

                # Se reproduce la canci\u00f3n
                searchResults = self.sp.search(song,1,0,"track")
                track = searchResults['tracks']['items'][0]
                self.sp.start_playback(uris=[track['uri']])

                self._set_color(COLORS_RGB['off'])
            elif inference.intent == 'album':
                # Se pasa la reproduccion
                self.sp.pause_playback() 
                
                # Se pregunta al usuario el album
                voice = AudioSegment.from_mp3("album.mp3")
                play(voice)

                # Se obtiene el album
                album = self._speech_to_text()  

                # Se reproduce el album
                searchResults = self.sp.search(album,1,0,"album")
                album = searchResults['albums']['items'][0]
                self.sp.start_playback(context_uri=album['uri'])
                
                self._set_color(COLORS_RGB['off'])
            elif inference.intent == 'artista':
                # Se pasa la reproduccion
                self.sp.pause_playback() 

                # Se pregunta al usuario el artista
                voice = AudioSegment.from_mp3("artista.mp3")
                play(voice)

                # Se obtiene el artista
                artist = self._speech_to_text()  

                # Se reproduce el artista
                searchResults = self.sp.search(artist,1,0,"artist")
                artist = searchResults['artists']['items'][0]
                self.sp.start_playback(context_uri=artist['uri'])
                
                self._set_color(COLORS_RGB['off'])
            elif inference.intent == 'playlist':

                # Se pasa la reproduccion
                self.sp.pause_playback()
                
                # Se pregunta la playlist
                voice = AudioSegment.from_mp3("playlist.mp3")
                play(voice)

                # Se obtiene la playlist
                playlist = self._speech_to_text()  

                # Se reproduce la playlist
                searchResults = self.sp.search(playlist,1,0,"playlist")
                playlist = searchResults['playlists']['items'][0]
                self.sp.start_playback(context_uri=playlist['uri'])

                self._set_color(COLORS_RGB['off'])
            elif inference.intent == 'playlistRec': 
                # Se pausa la reproduccion
                self.sp.pause_playback() 
                
                # Se pregunta como se encuentra 
                voice = AudioSegment.from_mp3("como_estas.mp3")
                play(voice)

                # Analisis de texto
                text = self._speech_to_text()
                document = language.Document(content=text, type_=language.Document.Type.PLAIN_TEXT)

                # Detecta la emocion del texto
                sentiment = self.nl.analyze_sentiment(request={'document': document}).document_sentiment

                print("Text: {}".format(text))
                print("Sentiment: {}, {}".format(sentiment.score, sentiment.magnitude))

                # Se categorizan las emociones y se reproduce una lista de reproduccion
                if sentiment.score > 0.25:
                    # positiva
                    print("Emocion positiva")
                    self._set_color(COLORS_RGB['red'])
                    voice = AudioSegment.from_mp3("positiva.mp3")
                    play(voice)
                    self.sp.start_playback(context_uri="spotify:playlist:37i9dQZF1DX9Dh2wgiAwVX")
                    self._set_color(COLORS_RGB['off'])
                    
                elif sentiment.score < 0.0:
                    # negativa
                    print("Emocion negativa")
                    self._set_color(COLORS_RGB['blue'])
                    voice = AudioSegment.from_mp3("negativa.mp3")
                    play(voice)
                    self.sp.start_playback(context_uri="spotify:playlist:37i9dQZF1DXdZjf8WgcTKM")
                    self._set_color(COLORS_RGB['off'])
                    
                elif sentiment.score == 0.0 and sentiment.magnitude >= 4.0:
                    # mixta
                    print("Emocion mixta")
                    self._set_color(COLORS_RGB['yellow'])
                    voice = AudioSegment.from_mp3("mixta.mp3")
                    play(voice)
                    self.sp.start_playback(context_uri="spotify:playlist:37i9dQZF1DX7qRKBHjmYIE")
                    self._set_color(COLORS_RGB['off'])
                    
                else:
                    # neutra
                    print("Emocion neutra")
                    self._set_color(COLORS_RGB['white'])
                    voice = AudioSegment.from_mp3("neutra.mp3")
                    play(voice)
                    self.sp.start_playback(context_uri="spotify:playlist:37i9dQZF1DWV1PBrIG2weG") 
                    self._set_color(COLORS_RGB['off'])
           
            elif inference.intent == 'play':
                self.sp.start_playback()
                self._set_color(COLORS_RGB['off'])
            elif inference.intent == 'pause':
                    self.sp.pause_playback()
                    self._set_color(COLORS_RGB['off'])
            else:
                raise NotImplementedError()

    def run(self):
        pa = None
        audio_stream = None

        try:
            # Se abre un stream de audio
            pa = pyaudio.PyAudio()

            audio_stream = pa.open(
                rate=self._picovoice.sample_rate,
                channels=1,
                format=pyaudio.paInt16,
                input=True,
                frames_per_buffer=self._picovoice.frame_length)

            print(self._context)

            print('[Listening ...]')

            # Se procesan los frames de audio y se pasan al objeto picovoice
            while True:
                pcm = audio_stream.read(self._picovoice.frame_length)
                pcm = struct.unpack_from("h" * self._picovoice.frame_length, pcm)

                self._picovoice.process(pcm)
        except KeyboardInterrupt:
            sys.stdout.write('\b' * 2)
            print('Stopping ...')
        finally:
            power.close()
            if audio_stream is not None:
                audio_stream.close()

            if pa is not None:
                pa.terminate()

            self._picovoice.delete()

def main():
    # Se indica la ruta de los archivos de palabra clave y de contexto
    o = PicovoiceDemo('picovoice_raspberry-pi.ppn', 'Polaris_es_raspberry-pi_2021-05-15-utc_v1_6_0.rhn')
    o.run()

if __name__ == '__main__':
    main()